# Imports 

In [1]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -r Requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tranformers (from versions: none)
ERROR: No matching distribution found for tranformers


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Clean PolitiFact

In [2]:
import pandas as pd

df = pd.read_json(r'Datasets\PolitiFact\PolitiFact.json', lines=True)

In [3]:
df.head()

,verdict,statement_originator,statement,statement_date,statement_source,factchecker,factcheck_date,factcheck_analysis_link
0,true,Barack Obama,John McCain opposed bankruptcy protections for...,6/11/2008,speech,Adriel Bettelheim,6/16/2008,https://www.politifact.com/factchecks/2008/jun...
1,false,Matt Gaetz,"""Bennie Thompson actively cheer-led riots in t...",6/7/2022,television,Yacob Reyes,6/13/2022,https://www.politifact.com/factchecks/2022/jun...
2,mostly-true,Kelly Ayotte,"Says Maggie Hassan was ""out of state on 30 day...",5/18/2016,news,Clay Wirestone,5/27/2016,https://www.politifact.com/factchecks/2016/may...
3,false,Bloggers,"""BUSTED: CDC Inflated COVID Numbers, Accused o...",2/1/2021,blog,Madison Czopek,2/5/2021,https://www.politifact.com/factchecks/2021/feb...
4,half-true,Bobby Jindal,"""I'm the only (Republican) candidate that has ...",8/30/2015,television,Linda Qiu,8/30/2015,https://www.politifact.com/factchecks/2015/aug...


In [4]:
df.shape

(21152, 8)

In [5]:
df.verdict.value_counts()

verdict
false           5625
half-true       3597
mostly-false    3432
mostly-true     3332
pants-fire      2703
true            2463
Name: count, dtype: int64

In [ ]:
# Binarise the labels
binary_map = {
    'true': 1,
    'mostly-true': 1, 
    'half-true': 1,
    'mostly-false': 0,
    'false': 0,
    'pants-fire': 0 
}

df['binary_verdict'] = df['verdict'].map(binary_map)

In [ ]:
import re

# Clean the dataset
def clean_text(text):
    '''Make text lowercase, remove punctuation and remove words containing numbers.'''
    # text = text.lower()
    # text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # text = re.sub('\w*\d\w*', '', text)
    # text = re.sub('\n', '', text)
    # text = re.sub('[0-9]+', '', text)
    # text = re.sub(r" +", " ", text)
    text = re.sub('\u00a0', ' ', text)
    text = re.sub('\u2019', '\'', text)
    text = re.sub('\u2018', '\'', text)
    # text = re.sub('\u2019', '', text)
    # text = re.sub('\u2018', '', text)
    # text = re.sub('\u2026', '', text)
    # text = re.sub('\u2013', '', text)
    return text

def remove_spaces(text):
    text = re.sub(r" +", " ", text)
    return text

In [ ]:
df['cleaned_statement'] = df['statement'].apply(clean_text)
#df['cleaned_statement'] = df['cleaned_statement'].apply(remove_spaces)

In [35]:
df.head()

,verdict,statement_originator,statement,statement_date,statement_source,factchecker,factcheck_date,factcheck_analysis_link,binary_verdict,cleaned_statement
0,true,Barack Obama,John McCain opposed bankruptcy protections for...,6/11/2008,speech,Adriel Bettelheim,6/16/2008,https://www.politifact.com/factchecks/2008/jun...,1,John McCain opposed bankruptcy protections for...
1,false,Matt Gaetz,"""Bennie Thompson actively cheer-led riots in t...",6/7/2022,television,Yacob Reyes,6/13/2022,https://www.politifact.com/factchecks/2022/jun...,0,"""Bennie Thompson actively cheer-led riots in t..."
2,mostly-true,Kelly Ayotte,"Says Maggie Hassan was ""out of state on 30 day...",5/18/2016,news,Clay Wirestone,5/27/2016,https://www.politifact.com/factchecks/2016/may...,1,"Says Maggie Hassan was ""out of state on 30 day..."
3,false,Bloggers,"""BUSTED: CDC Inflated COVID Numbers, Accused o...",2/1/2021,blog,Madison Czopek,2/5/2021,https://www.politifact.com/factchecks/2021/feb...,0,"""BUSTED: CDC Inflated COVID Numbers, Accused o..."
4,half-true,Bobby Jindal,"""I'm the only (Republican) candidate that has ...",8/30/2015,television,Linda Qiu,8/30/2015,https://www.politifact.com/factchecks/2015/aug...,1,"""I'm the only (Republican) candidate that has ..."


In [4]:
from torch.utils.data import Dataset, DataLoader, random_split

# get the sample dataset
class PolitifactDataset(Dataset):
    def __init__(self, text, originator, labels):
        self.labels = labels.values
        self.text = text.values
        self.originator = originator.values
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]
        originator = self.originator[idx] 

        sample = (label, text, originator)
        return sample

In [5]:
dataset_label = PolitifactDataset(df["statement"],
                            df["statement_originator"],
                            df["binary_verdict"])

In [ ]:
# Print the first element
first_element = next(iter(dataset_label))

first_element = next(iter(dataset_label))
print(f"Originator: {first_element[2]}")
print(f"Text: {first_element[1]}")
print(f"Label: {first_element[0]}\n")

# Length of dataset
print('Length of dataset: ', len(dataset_label), '\n')


Originator: Barack Obama
Text: John McCain opposed bankruptcy protections for families "who were only in bankruptcy because of medical expenses they couldn't pay."
Label: 1

Length of dataset:  21152 



In [ ]:
import json

# Convert dataset_label into a JSON serialisable format
dataset_json = [
    {"statement": statement, "originator": originator, "label": label}
    for statement, originator, label in zip(df["statement"], df["statement_originator"], df["binary_verdict"])
]

# Save as JSON file
json_filename = r'Datasets\PolitiFact\PolitiFact_Clean.json'
with open(json_filename, "w", encoding="utf-8") as json_file:
    json.dump(dataset_json, json_file, indent=4, ensure_ascii=False)

print(f"✅ Dataset saved as {json_filename}")


✅ Dataset saved as Datasets\PolitiFact\PolitiFact_Clean.json


# Dataset For Fine-tuning

In [ ]:
import json

# Load full dataset
with open(r'Datasets\PolitiFact\PolitiFact_Clean.json', 'r', encoding="utf-8") as f:
    data_ft = json.load(f)

# Convert into desired format
converted_data = []
for entry in data_ft:
    text = f'{entry["originator"]} says: {entry["statement"]}'
    label = entry["label"]
    converted_data.append({
        "text": text,
        "label": label
    })

# Save to JSONL
with open(r'Datasets\PolitiFact\PolitiFact_Fine_Tuning.jsonl', 'w', encoding="utf-8") as f:
    for entry in converted_data:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')

print("Saved PolitiFact_Fine_Tuning.jsonl.")

Saved PolitiFact_Fine_Tuning.jsonl.


In [ ]:
import json

# Load existing datapoint dataset
with open(r"Datasets\PolitiFact\PolitiFact_Fine_Tuning.jsonl", "r", encoding="utf-8") as f:
    original_data = [json.loads(line) for line in f]

# Convert into prompt-completion format, first 5000 entries
prompt_completion_data = []
for entry in original_data[:5000]:
    text = entry["text"]
    label = "True" if entry["label"] == 1 else "False"

    prompt = f"'{text}'\nIs this claim True or False?"
    completion = f"The claim is {label}"

    prompt_completion_data.append({"prompt": prompt, "completion": completion})

# Save the new format
with open(r"Datasets\PolitiFact\PolitiFact_PromptCompletion.jsonl", "w", encoding="utf-8") as f:
    for entry in prompt_completion_data:
        f.write(json.dumps(entry) + "\n")

print("✅ Dataset converted to prompt-completion format.")

✅ Dataset converted to prompt-completion format.


# Test

In [ ]:
import json

# Load your existing datapoint dataset
with open(r"Datasets\PolitiFact\PolitiFact_Fine_Tuning.jsonl", "r", encoding="utf-8") as f:
    original_data = [json.loads(line) for line in f]

# Convert into prompt-completion format entries 5000-6000
prompt_completion_data = []
for entry in original_data[5000:6000]:
    text = entry["text"]
    label = "True" if entry["label"] == 1 else "False"

    prompt = f"'{text}'\nIs this claim True or False?"
    completion = f"{label}"

    prompt_completion_data.append({"prompt": prompt, "completion": completion})

# Save the new format
with open(r"Datasets\PolitiFact_PromptCompletion_Test.jsonl", "w", encoding="utf-8") as f:
    for entry in prompt_completion_data:
        f.write(json.dumps(entry) + "\n")

print("✅ Dataset converted to prompt-completion format.")

✅ Dataset converted to prompt-completion format.


In [11]:
import json
from collections import Counter

with open(r'Datasets\PolitiFact_PromptCompletion_Test.jsonl', 'r', encoding="utf-8") as f:
    datas = [json.loads(line) for line in f]

datas = datas[:100]
# Extract labels and count
label_counts = Counter(entry['completion'] for entry in datas)
print(label_counts)


Counter({'False': 55, 'True': 45})


In [ ]:
import json
import re

# Define input and output file paths
input_path = r"Datasets\PolitiFact\PolitiFact_PromptCompletion.jsonl"
output_path = r"Datasets\PolitiFact_Alpaca.json"

# Load the dataset from JSONL
with open(input_path, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]

# Convert dataset to Alpaca format
alpaca_dataset = []

for entry in dataset:
    # Extract the full prompt and completion
    full_prompt = entry.get("prompt", "").strip()
    completion = entry.get("completion", "").strip()

    # Split the full prompt into "instruction" & "input"
    if "\n" in full_prompt:  
        instruction, input_text = full_prompt.split("\n", 1)
        input_text = input_text.strip()
    else:
        instruction = full_prompt  
        input_text = ""

    # Remove unnecessary surrounding quotes from instruction
    instruction = re.sub(r"^['\"]|['\"]$", "", instruction).strip()

    # Create an Alpaca-style dictionary
    alpaca_entry = {
        "instruction": instruction,
        "input": input_text,
        "output": completion
    }

    alpaca_dataset.append(alpaca_entry)

# Save the new dataset as JSON
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(alpaca_dataset, f, indent=4, ensure_ascii=False)

print(f"✅ Dataset converted to Alpaca format and saved at: {output_path}")

✅ Dataset converted to Alpaca format and saved at: Datasets\PolitiFact_Alpaca.json


In [ ]:
import json
import re

# Define input and output file paths
input_path = r"Datasets\PolitiFact_PromptCompletion_Test.jsonl"
output_path = r"Datasets\PolitiFact_Alpaca_Test.json"

# Load the dataset from JSONL
with open(input_path, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]

# Convert dataset to Alpaca format
alpaca_dataset = []

for entry in dataset:
    # Extract the full prompt and completion
    full_prompt = entry.get("prompt", "").strip()
    completion = entry.get("completion", "").strip()

    # Split the full prompt into "instruction" & "input"
    if "\n" in full_prompt:  
        instruction, input_text = full_prompt.split("\n", 1)
        input_text = input_text.strip()
    else:
        instruction = full_prompt  
        input_text = ""

    # Remove unnecessary surrounding quotes from instruction
    instruction = re.sub(r"^['\"]|['\"]$", "", instruction).strip()

    # Create an Alpaca-style dictionary
    alpaca_entry = {
        "instruction": instruction,
        "input": input_text,
        "output": completion
    }

    alpaca_dataset.append(alpaca_entry)

# Save the new dataset as JSON
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(alpaca_dataset, f, indent=4, ensure_ascii=False)

print(f"✅ Dataset converted to Alpaca format and saved at: {output_path}")

✅ Dataset converted to Alpaca format and saved at: Datasets\PolitiFact_Alpaca_Test.json
